In [228]:
import instascrape
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse
from datastructures import Location, Site
from dataclasses_serialization.json import JSONSerializer
import time
import json

driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Driver [/home/matas/.wdm/drivers/chromedriver/linux64/90.0.4430.24/chromedriver] found in cache


# 1. Get a list of all Locations that Instagram supports

In [229]:
instagram_us_cities = "https://www.instagram.com/explore/locations/US/united-states/"
driver.get(instagram_us_cities)
soup = BeautifulSoup(driver.page_source)

data = {}

next_page = not None

while next_page is not None:
    next_page = soup.find("a", href=re.compile("\?page="))

    links = soup.find_all("a", href=re.compile("/explore/locations/"))
    for link in links:
        if link.text == "See More":
            continue
        data[link.text] = Location(link.text, link["href"], [])

    if next_page is None:
        break

    print("Getting the next page:", next_page["href"])
    driver.get(urllib.parse.urljoin(base_url, next_page["href"]))
    soup = BeautifulSoup(driver.page_source)
    time.sleep(2)

# Remove last link to more locations
del data["Locations"]

Getting the next page: /explore/locations/US/united-states/?page=2
Getting the next page: /explore/locations/US/united-states/?page=3
Getting the next page: /explore/locations/US/united-states/?page=4
Getting the next page: /explore/locations/US/united-states/?page=5
Getting the next page: /explore/locations/US/united-states/?page=6
Getting the next page: /explore/locations/US/united-states/?page=7
Getting the next page: /explore/locations/US/united-states/?page=8
Getting the next page: /explore/locations/US/united-states/?page=9
Getting the next page: /explore/locations/US/united-states/?page=10
Getting the next page: /explore/locations/US/united-states/?page=11


# Get the Sites from each Location

In [243]:
for key, location in data.items():
    print(location.name)
    break

Midtown East


## Serialize / Deserialize the data

In [244]:
def serialize_instagram_data(data: dict, path: str) -> None:
    with open(path, "w") as file:
        json.dump(JSONSerializer.serialize(data), file)

def deserialize_instagram_data(path: str) -> dict:
    data = json.loads(open(path, "r").read())
    for key, val in data.items():
        data[key] = JSONSerializer.deserialize(Location, data[key])
    return data